In [87]:
import http.client, urllib.request, urllib.parse, urllib.error, base64
import requests
import pandas as pd
from io import StringIO
from io import BytesIO
import json
import numpy as np

In [2]:
researchers = pd.read_csv('HDSI Faculty Council - Sheet1 - HDSI Faculty Council - Sheet1.csv')

In [3]:
author_name = [x.lower() for x in list(researchers['Name'])]

In [37]:
ms_id = [2243367769, 2529224718] # Justin, Aaron

In Microsoft Research API, we also follow the same way of getting author id from urls 

# Difference between Microsoft and Dimensions
* In MS, no concepts (but have a list of field of study), date (only have year)
* there is only unique keywords of abstracts in microsoft dataset 
* some papers do not have abstracts keywords (AW)
* some papers do not have publisher (PB) & journal (J) 
* we can find Justin (8 publications) and Aaron (1 from 2013) in the dataset

# Data Extraction and Aggregation

In [38]:
# Data Extraction From MS
data_ms = []

headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': '93e9736ec66846b5a3252e08f3ec5f4c',
}

for id in ms_id:
    params = urllib.parse.urlencode({
        # Request parameters
        'expr': "AND(Composite(AA.AuId={}))".format(id),
        'model': 'latest',
        'count': '9999',
        'offset': '0',
        # 'orderby': '{string}',
        'attributes': 'Id,AW,DN,F.DFN,Y,J.JN,CC,AA.DAuN',
    })

    try:
        conn = http.client.HTTPSConnection('api.labs.cognitive.microsoft.com')
        conn.request("GET", "/academic/v1.0/evaluate?%s" % params, "{body}", headers)
        response = conn.getresponse()
        bytes_data = response.read()
        data_ms.append(bytes_data)
        # print(data)
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))

In [39]:
data = []

In [40]:
for i in range(len(data_ms)):
    my_json = data_ms[i].decode('utf8')
    data.append(json.loads(my_json))

In [47]:
# Organize the data in usable format

wd = dict()
for j in range(len(data)):
    # author id
    author = data[j]['expr'][22:-2]

    # for each author
    d = dict()

    for i in range(len(data[j]['entities'])):
        # paper id as keys
        id_key = str(data[j]['entities'][i]['Id'])
        if id_key not in d.keys():
            d[id_key] = {}

        # abstract inner keys
        if "abstract" not in d[id_key].keys():
            d[id_key]['abstract'] = []
        if "AW" not in data[j]['entities'][i].keys():
            d[id_key]['abstract'] = []
        else:
            d[id_key]['abstract'] += data[j]['entities'][i]['AW']
            d[id_key]['abstract'] = list(set(d[id_key]['abstract']))
        
        # title inner keys
        d[id_key]['title'] = data[j]['entities'][i]['DN']

        # field inner keys
        d[id_key]['field'] = data[j]['entities'][i]['F']

        #year inner key
        d[id_key]['year'] = data[j]['entities'][i]['Y']

        if "J" not in data[j]['entities'][i].keys():
            d[id_key]['journal'] = {}
        else:
            d[id_key]['journal'] = data[j]['entities'][i]['J']

        d[id_key]['times_cited'] = data[j]['entities'][i]['CC']

        d[id_key]['author_name'] = data[j]['entities'][i]['AA']
        

    
    wd['{}'.format(author)] = d


In [49]:
# justin
df1 = pd.DataFrame.from_dict(wd['2243367769'], orient='index')

In [51]:
# aaron
df2 = pd.DataFrame.from_dict(wd['2529224718'], orient='index')
df2

,abstract,title,field,year,journal,times_cited,author_name
1895840270,"[characterize, extensions, pi, singular, alway...",Extensions of Poisson Structures on Singular H...,"[{'DFN': 'Isolated singularity'}, {'DFN': 'Kos...",2013,{'JN': 'arxiv symplectic geometry'},1,[{'DAuN': 'Aaron McMillan Fraenkel'}]


In [78]:
df = pd.concat([df1,df2])

In [81]:
df

,abstract,title,field,year,journal,times_cited,author_name
2963037546,"[weyl, algorithm, typical, davis, unperturbed,...",Unperturbed: spectral analysis beyond Davis-Kahan,"[{'DFN': 'Perturbation theory'}, {'DFN': 'Eige...",2018,{},50,"[{'DAuN': 'Justin Eldridge'}, {'DAuN': 'Mikhai..."
2648734036,"[weyl, algorithm, typical, davis, unperturbed,...",Unperturbed: spectral analysis beyond Davis-Kahan,"[{'DFN': 'Eigenvalues and eigenvectors'}, {'DF...",2017,{'JN': 'arxiv machine learning'},9,"[{'DAuN': 'Justin Eldridge'}, {'DAuN': 'Mikhai..."
2963918728,"[correct, imply, single, nesting, two, imprope...",Beyond Hartigan Consistency: Merge Distortion ...,"[{'DFN': 'Hierarchical clustering'}, {'DFN': '...",2015,{},26,"[{'DAuN': 'Justin Eldridge'}, {'DAuN': 'Mikhai..."
1963146503,"[correct, imply, single, nesting, two, imprope...",Beyond Hartigan Consistency: Merge Distortion ...,"[{'DFN': 'Hierarchical clustering'}, {'DFN': '...",2015,{'JN': 'arxiv machine learning'},1,"[{'DAuN': 'Justin Eldridge'}, {'DAuN': 'Mikhai..."
2963004507,"[correct, assumption, algorithm, produce, grap...","Graphons, mergeons, and so on!","[{'DFN': 'Clustering coefficient'}, {'DFN': 'C...",2016,{},13,"[{'DAuN': 'Justin Eldridge'}, {'DAuN': 'Mikhai..."
2460367511,"[correct, assumption, algorithm, produce, grap...","Graphons, mergeons, and so on!","[{'DFN': 'Clustering coefficient'}, {'DFN': 'C...",2016,{'JN': 'arxiv machine learning'},0,"[{'DAuN': 'Justin Eldridge'}, {'DAuN': 'Mikhai..."
2004669647,"[hyper, recordings, noise, means, taken, show,...",Robust features for the automatic identificati...,"[{'DFN': 'Electroencephalography'}, {'DFN': 'A...",2014,{'JN': 'journal of neurodevelopmental disorders'},26,"[{'DAuN': 'Justin Eldridge'}, {'DAuN': 'Alison..."
2276719562,[],Robust features for the automatic identificati...,"[{'DFN': 'Autism spectrum disorder'}, {'DFN': ...",2014,{},0,"[{'DAuN': 'Justin Eldridge'}, {'DAuN': 'Alison..."
1895840270,"[characterize, extensions, pi, singular, alway...",Extensions of Poisson Structures on Singular H...,"[{'DFN': 'Isolated singularity'}, {'DFN': 'Kos...",2013,{'JN': 'arxiv symplectic geometry'},1,[{'DAuN': 'Aaron McMillan Fraenkel'}]


In [96]:
df['authors'] = df['author_name'].apply(lambda x: [x[i]['DAuN'] for i in range(len(x))])
df['authors_count'] = df['authors'].apply(lambda x: len(x))
df['concepts'] = df['field'].apply(lambda x: [x[i]['DFN'] for i in range(len(x))])
df['journal.title'] = df['journal'].apply(lambda x: x['JN'] if len(x) > 0 else np.NaN)

In [99]:
df = df.drop(['field','journal','author_name'],axis=1)

In [101]:
df.to_csv('justin&aaron.csv')